In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os


import gc
import pickle as pickle

from sklearn.model_selection import KFold

from catboost import CatBoostRegressor
import lightgbm as lgbm

import itertools

import multiprocessing as mp
import importlib
from sklearn.model_selection import train_test_split

In [3]:
#tail -n +2 train.csv | split -l 150000


In [4]:
from os import listdir
from os.path import isfile, join

TRAIN_SPLITS='train'
splits = [f for f in listdir(TRAIN_SPLITS) if isfile(join(TRAIN_SPLITS, f))]

In [5]:
TIMESTEPS=1

import build_segment
importlib.reload(build_segment)

from build_segment import build_segment_f

split_chunks = np.array_split(splits,mp.cpu_count())

#def build_segment_f(splits, number_of_groups,test=False, augment=False, scale=True, noise=0.5):

param_test = False
param_augment = False
param_scale = False
param_noise = 0.5


if __name__ ==  '__main__':
    pool = mp.Pool(mp.cpu_count())
    res = [pool.apply_async(build_segment_f,args=[chunk,TIMESTEPS,param_test,param_augment,param_scale,param_noise]) for chunk in split_chunks]
    pool.close()
    pool.join()

0
0
0
0
0
0
0
0
100
100
100
100
100
100
100
100
200
200
200
200
200
200
200
200
300
300
300
300
300
300
300
300
400
400
400
400
400
400
400
400
500
500
500
500
500
500
500
500


In [ ]:
res[0].get()[100]

In [6]:
columns=res[0].get()[0].columns.values

data = pd.DataFrame(columns=columns)


i=0
for r in res:
    for df in r.get():
        data = data.append(df)
        i+=1
        

data.reset_index(drop=True,inplace=True)


In [ ]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
data.sample(10)

In [7]:
combinations2 = list(itertools.combinations(data.columns.values, 2))
combinations3 = list(itertools.combinations(data.columns.values, 3))


In [8]:

for combination in combinations2:
    if 'augmented' in combination or 'time_to_failure' in combination:
        continue
    f1 = combination[0]
    f2 = combination[1]
    feature = f'{f1}_mult_{f2}'
    data[feature] = data[f1] * data[f2]
    feature = f'{f1}_plus_{f2}'
    data[feature] = data[f1] + data[f2]
    feature = f'{f1}_div_{f2}'
    #print(feature)
    data[feature] = data[f1] / data[f2]
    data[feature] = pd.to_numeric(data[feature], downcast='float')


In [9]:

for combination in combinations3:
    if 'augmented' in combination or 'time_to_failure' in combination:
        continue
    f1 = combination[0]
    f2 = combination[1]
    f3 = combination[2]
    feature = f'{f1}_mult_{f2}_mult_{f3}'
    data[feature] = data[f1] * data[f2] * data[f3]
    feature = f'{f1}_plus_{f2}_plus_{f3}'
    data[feature] = data[f1] + data[f2] + data[f3]
    feature = f'{f1}_div_{f2}_div_{f3}'
    #print(feature)
    data[feature] = data[f1] / data[f2] / data[f3]
    data[feature] = pd.to_numeric(data[feature], downcast='float')
    

In [10]:
len(combinations)

NameError: name 'combinations' is not defined

In [11]:

test_size = 0.1
#train_data, val_data, y_train, y_val = train_test_split(training, targets, test_size=test_size, random_state=42)

val_size = int(len(data)*0.1)
non_augmented = data[data['augmented'] == False]
val_indices = np.random.choice(non_augmented.index.values, val_size)
val_data = data[data.index.isin(val_indices)].drop('augmented', axis=1)
train_data = data[~data.index.isin(val_indices)].drop('augmented', axis=1)

train_y = train_data['time_to_failure']
val_y = val_data['time_to_failure']

train_data = train_data.drop('time_to_failure', axis=1)
val_data = val_data.drop('time_to_failure', axis=1)

In [ ]:
len(val_y)

In [12]:
features = train_data.columns.values

In [ ]:
train_data.sample(10)

In [ ]:
list(features)

In [ ]:

cb_model = CatBoostRegressor(iterations=10000,
                             #learning_rate=0.05,
                             eval_metric='MAE',
                             task_type = "GPU",
                             use_best_model=True,
                             od_type = "Iter",
                             od_wait = 20,
                             #depth=3,
                             border_count=254,
                             #bagging_temperature = 20,
                             #cat_features=[0],
                             random_seed = 42)



cb_model.fit(train_data[features], train_y, #cat_features=categorical_features_indices,
             eval_set=(val_data[features],val_y),
             #cat_features=categorical_features_pos,         
             verbose=True)

scores = {}
for i,score in enumerate(cb_model.get_feature_importance()):
    scores[features[i]] = score

sorted(scores.items(), key=lambda x: x[1])[::-1]

In [17]:
lgtrain = lgbm.Dataset(train_data[features], label=train_y)
lgval = lgbm.Dataset(val_data[features], label=val_y)


# needs to keep overfiting under control, default params don't cut it at all
params = {
        "num_threads": 8,
        "verbosity": -1,
        #"zero_as_missing": "true",
        "boosting":'gbdt',
        "objective" : "regression",
        "metric" : "mae",
        "seed": 42,
        "learning_rate" : 0.05,
        #"min_data_in_leaf": 100,
        #"num_leaves": 10,
        #"max_depth" : 4,
        #"bagging_fraction": 0.7,
        #"bagging_freq": 1,
        #"feature_fraction": 0.7,
        #"lambda_l1": 10,
}

evals_result = {}
model_lgb = lgbm.train(params, lgtrain, 10000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=30, 
                      verbose_eval=10, 
                      evals_result=evals_result)


scores = {}
for i,score in enumerate(model_lgb.feature_importance()):
    scores[features[i]] = score

sorted(scores.items(), key=lambda x: x[1])[::-1]

Training until validation scores don't improve for 30 rounds.
[10]	valid_0's l1: 2.56877
[20]	valid_0's l1: 2.32894
[30]	valid_0's l1: 2.20539
[40]	valid_0's l1: 2.14617
[50]	valid_0's l1: 2.12913
[60]	valid_0's l1: 2.12064
[70]	valid_0's l1: 2.12249
[80]	valid_0's l1: 2.12429
Early stopping, best iteration is:
[57]	valid_0's l1: 2.12008


[('mean_mult_sum_mult_negs', 189),
 ('mean_div_pos', 135),
 ('imax_div_imin', 90),
 ('mean_mult_negs_mult_cid_ce', 88),
 ('max_div_min_div_ssum', 87),
 ('q75_div_autocorr_10_div_r_sigma', 80),
 ('min_plus_std_plus_uniq', 80),
 ('max_div_min_div_r_sigma', 74),
 ('abs_sum_chg_mult_cid_ce', 71),
 ('std_div_q50_div_negs', 68),
 ('min_div_abs_div_reocurring_pct', 68),
 ('q25_mult_imin_mult_autocorr_10', 66),
 ('mean_div_pos_div_abs_sum_chg', 64),
 ('kurtosis_div_r_sigma_div_skewness', 63),
 ('max_div_kurtosis_div_r_sigma', 63),
 ('negs_div_r_sigma_div_ratio_to_length', 61),
 ('max_div_min_div_imin', 57),
 ('negs_div_autocorr_10_div_reocurring_pct', 54),
 ('negs_div_autocorr_10_div_r_sigma', 53),
 ('negs_div_abs_nrg_div_r_sigma', 53),
 ('q50_div_imin_div_skewness', 53),
 ('min_div_abs_div_ssum', 47),
 ('q75_mult_negs_mult_autocorr_10', 46)]

In [16]:
features = [item[0] for item in scores.items() if item[1] > 60]

In [30]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression,BayesianRidge, HuberRegressor

lr_features = features[0:10]

lr_model =  Lasso(alpha=0.5)
lr_model.fit(train_data[lr_features], train_y)

from sklearn.metrics import mean_absolute_error

train_preds = lr_model.predict(val_data[lr_features])
print("mae: ", mean_absolute_error(val_y, train_preds))

mae:  2.1884148717136336


In [32]:
TEST_SPLITS='test'
test_splits = [f for f in listdir(TEST_SPLITS) if isfile(join(TEST_SPLITS, f))]

In [33]:
test_split_chunks = np.array_split(test_splits,mp.cpu_count())
    
    
param_test = True
param_augment = False
param_scale = False
param_noise = 0.6


if __name__ ==  '__main__':
    pool = mp.Pool(mp.cpu_count())
    res = [pool.apply_async(build_segment_f,args=[chunk,TIMESTEPS,param_test,param_augment,param_scale,param_noise]) for chunk in test_split_chunks]
    pool.close()
    pool.join()

0
0
0
0
0
0
0
0
100
100
100
100
100
100
100
100
200
200
200
200
200
200
200
200
300
300
300
300
300
300
300
300


In [35]:
columns=res[0].get()[0].columns.values

test_data = pd.DataFrame(columns=columns)


i=0
for r in res:
    for df in r.get():
        #print(df)
        test_data = test_data.append(df)
        #print(len(test_data))
        i+=1
        

test_data.reset_index(drop=True,inplace=True)
print(len(test_data))

2624


In [36]:
combinations2 = list(itertools.combinations(test_data.columns.values, 2))
combinations3 = list(itertools.combinations(test_data.columns.values, 3))


In [38]:

for combination in combinations2:
    if 'seg_id' in combination or 'time_to_failure' in combination:
        continue
    f1 = combination[0]
    f2 = combination[1]
    feature = f'{f1}_mult_{f2}'
    test_data[feature] = test_data[f1] * test_data[f2]
    feature = f'{f1}_plus_{f2}'
    test_data[feature] = test_data[f1] + test_data[f2]
    feature = f'{f1}_div_{f2}'
    #print(feature)
    test_data[feature] = test_data[f1] / test_data[f2]
    test_data[feature] = pd.to_numeric(test_data[feature], downcast='float')


In [39]:

for combination in combinations3:
    if 'seg_id' in combination or 'time_to_failure' in combination:
        continue
    f1 = combination[0]
    f2 = combination[1]
    f3 = combination[2]
    feature = f'{f1}_mult_{f2}_mult_{f3}'
    test_data[feature] = test_data[f1] * test_data[f2] * test_data[f3]
    feature = f'{f1}_plus_{f2}_plus_{f3}'
    test_data[feature] = test_data[f1] + test_data[f2] + test_data[f3]
    feature = f'{f1}_div_{f2}_div_{f3}'
    #print(feature)
    test_data[feature] = test_data[f1] / test_data[f2] / test_data[f3]
    test_data[feature] = pd.to_numeric(test_data[feature], downcast='float')
    

In [41]:
ids = test_data['seg_id'].apply(lambda id: id.split('.')[0])

In [ ]:
preds = cb_model.predict(test_data[features])


In [42]:
lg_preds = model_lgb.predict(test_data[features])

In [44]:
lr_preds = lr_model.predict(test_data[lr_features])

In [49]:
preds = (lg_preds * 0.5) + (lr_preds * 0.5)


In [52]:

submission = pd.DataFrame(ids)
submission.columns = ['seg_id']
submission['time_to_failure'] = lg_preds

submission.to_csv('submission.csv', index=False)

In [53]:
submission["time_to_failure"].describe()

count    2624.000000
mean        5.863660
std         2.288841
min         1.269167
25%         4.052866
50%         5.483414
75%         7.649506
max        11.784076
Name: time_to_failure, dtype: float64

In [48]:
submission["time_to_failure"].describe()

count    2624.000000
mean        5.347878
std         2.108800
min        -4.808612
25%         3.805956
50%         4.916270
75%         6.885264
max        10.936613
Name: time_to_failure, dtype: float64

In [ ]:
submission.head()

In [ ]:
lgbm_preds = pd.read_csv('lgbm.csv')['time_to_failure']
cb_preds = pd.read_csv('cb1549.csv')['time_to_failure']
lstm_preds = pd.read_csv('lstm1627.csv')['time_to_failure']


preds = (cb_preds * 0.4) + (lgbm_preds * 0.5)

preds.clip(0,20,out=preds)

print(np.mean(preds))
print(np.max(preds))

submission = test.loc[:,['ID']]
submission['item_cnt_month'] = preds

submission.to_csv('submission.csv', index=False)